In [1]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np
import tensorflow as tf
import random
import matplotlib.pyplot as plt
import cv2
import os

2024-11-29 17:02:30.961813: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-29 17:02:30.971825: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-29 17:02:30.999151: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732879051.039703   54728 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732879051.051828   54728 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-29 17:02:31.099225: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
img_size = 32

In [3]:
this = []

_root_working = "/home/rohan/Projects/ML/MRI/DataSets/New_TUMOR/glioma"
for _lo in os.listdir(_root_working):
    _lo = os.path.join(_root_working, _lo)
    _lo = cv2.imread(_lo)
    _lo = cv2.resize(_lo, (img_size, img_size)) / 255
    this.append(_lo)

In [4]:
this = np.array(this)

In [5]:
that = []

_root = "/home/rohan/Projects/ML/MRI/DataSets/New_TUMOR/"
for _lo in os.listdir(_root):
    _lo = os.path.join(_root, _lo)
    if _lo == _root_working:
        continue
    for _nj in os.listdir(_lo):
        _nj = os.path.join(_lo, _nj)
        _nj = cv2.imread(_nj)
        _nj = cv2.resize(_nj, (img_size, img_size)) / 255
        that.append(_nj)

In [6]:
that = np.array(that)

In [7]:
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(img_size, img_size, 3)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),

    Flatten(),

    Dense(128, activation="relu"),

    Dropout(.5),

    Dense(1, activation="sigmoid"),
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-11-29 17:02:44.094088: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [8]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
x, y = np.concatenate([this, that], axis=0), np.concatenate([np.ones(this.shape[0]), np.zeros(that.shape[0])], axis=0)

In [15]:
model.fit(x, y, epochs=5, batch_size=32)

Epoch 1/5
105/105 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.9890 - loss: 0.0329
Epoch 2/5
105/105 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.9841 - loss: 0.0426
Epoch 3/5
105/105 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.9893 - loss: 0.0295
Epoch 4/5
105/105 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.9951 - loss: 0.0187
Epoch 5/5
105/105 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.9933 - loss: 0.0199


In [16]:
model.save(f"/home/rohan/Projects/ML/MRI/model/{_root_working.split('/')[-1]}.keras")

In [12]:
def test_this_image(path):
    img = cv2.imread(path)
    img = cv2.resize(img, (img_size, img_size))
    img = np.expand_dims(img, axis=0)
    return model.predict(img)

In [13]:
test_this_image("/home/rohan/Projects/ML/MRI/DataSets/Sclerosis/IM000015.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step


array([[0.]], dtype=float32)